In [1]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [2]:
import json
import os
import json
import numpy as np
import pandas as pd
import re

In [3]:
data=pd.read_json('https://raw.githubusercontent.com/kajalpanda1/CDAC_Assignments/master/NLP/Assignment-3/sample_conversations.json')

In [4]:
data.head()

,NumTextMessages,Issues
0,22264,"{'IssueId': 1, 'CompanyGroupId': 1, 'Messages'..."
1,22264,"{'IssueId': 10001, 'CompanyGroupId': 1, 'Messa..."
2,22264,"{'IssueId': 20001, 'CompanyGroupId': 1, 'Messa..."
3,22264,"{'IssueId': 30001, 'CompanyGroupId': 40001, 'M..."
4,22264,"{'IssueId': 40001, 'CompanyGroupId': 50001, 'M..."


In [5]:
data.shape

(1508, 2)

In [6]:
for column in ['Issues']:
  column_as_df = pd.json_normalize(data[column])
  column_as_df.columns = [str(column+"_"+subcolumn) for subcolumn in column_as_df.columns]
  data = data.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
  df = pd.DataFrame([dict(y, index=i) for i, x in enumerate(data['Issues_Messages'].values.tolist()) for y in x])

In [7]:
df.shape

(22264, 3)

In [8]:
df.head()

,IsFromCustomer,Text,index
0,True,Hi! I placed an order on your website and I ca...,0
1,True,I think I used my email address to log in.,0
2,True,My battery exploded!,1
3,True,"It's on fire, it's melting the carpet!",1
4,True,What should I do!,1


In [55]:
#selecting representative texts
new_df=pd.DataFrame()
new_df=df[df.IsFromCustomer==False]

In [56]:
new_df.head()

,IsFromCustomer,Text,index
9,False,Hello Werner how may I help you today?,3
11,False,Sure I can help you with that? Could you pleas...,3
13,False,Let me update that information on our system,3
14,False,"OK Wernzio, I have updated your address to the...",3
16,False,Ok let me go ahead and request a work order fo...,3


In [57]:
new_df.reset_index()

,level_0,IsFromCustomer,Text,index
0,9,False,Hello Werner how may I help you today?,3
1,11,False,Sure I can help you with that? Could you pleas...,3
2,13,False,Let me update that information on our system,3
3,14,False,"OK Wernzio, I have updated your address to the...",3
4,16,False,Ok let me go ahead and request a work order fo...,3
...,...,...,...,...
11055,22219,False,Thank you. Good Bye,1495
11056,22251,False,"Sorry for the wait, I can help you .",1505
11057,22258,False,no problem. I can help,1506
11058,22260,False,Sorry to hear that I can help you with that.,1507


In [58]:
#splitting the dataframe and creating new rows
def splitDataFrameList(df,target_column,separator):

  def split_text(line, separator):
    splited_line =  [e+d for e in line.split(separator) if e]
    return splited_line
    
  def splitListToRows(row,row_accumulator,target_column,separator):
    split_row = row[target_column].split(separator)
    for s in split_row:
      new_row = row.to_dict()
      new_row[target_column] = s
      row_accumulator.append(new_row)
  new_rows = []
  df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
  new_df = pd.DataFrame(new_rows)
  return new_df

In [59]:
#splitting sentences on punctuations
for sep in ['. ',', ','? ', '! ', '; ']:
  new_df = splitDataFrameList(new_df, 'Text', sep)

In [60]:
new_df.head()

,IsFromCustomer,Text,index
0,False,Hello Werner how may I help you today?,3
1,False,Sure I can help you with that,3
2,False,Could you please provide me with your new addr...,3
3,False,Let me update that information on our system,3
4,False,OK Wernzio,3


In [61]:
new_df.shape

(19077, 3)

In [62]:
#text Cleaning using simple regex
new_df['Text']=new_df['Text'].apply(lambda x: " ".join(x.split()))
new_df['Text']=new_df['Text'].apply(lambda x: x.strip("."))
new_df['Text']=new_df['Text'].apply(lambda x: " ".join(x.split()))
new_df['Text']=new_df['Text'].apply(lambda x: x.replace(' i ',' I '))
new_df['Text']=new_df['Text'].apply(lambda x: x.replace(' ?','?'))
new_df['Text']=new_df['Text'].apply(lambda x: x.replace(' !','!'))
new_df['Text']=new_df['Text'].apply(lambda x: x.replace(' .','.'))
new_df['Text']=new_df['Text'].apply(lambda x: x[0].upper()+x[1:])
new_df['Text']=new_df['Text'].apply(lambda x: x+"?" if re.search(r'^(Wh|How).+([^?])$',x) else x)

In [63]:
#calculating nb_words in the sentence
new_df['nb_words'] = new_df['Text'].apply(lambda x: len(str(x).split(' ')))
new_df = new_df[new_df['nb_words']>2]

In [64]:
#count occurences of sentence
new_df['Counts'] = new_df.groupby(['Text'])['Text'].transform('count')

In [65]:
new_df.head()

,IsFromCustomer,Text,index,nb_words,Counts
0,False,Hello Werner how may I help you today?,3,8,1
1,False,Sure I can help you with that,3,7,1
2,False,Could you please provide me with your new addr...,3,9,9
3,False,Let me update that information on our system,3,8,1
5,False,I have updated your address to the system,3,8,1


In [66]:
#removing duplicates 
new_df = new_df.drop_duplicates(subset=['Text'], keep='last')

In [67]:
new_df.shape

(8574, 5)

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances

In [69]:
#creating tfidf matrix
model_tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 5), min_df=0)
tfidf = model_tf.fit_transform(new_df['Text'])

In [70]:
model_tf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=0, ngram_range=(1, 5), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [71]:
tfidf.shape

(8574, 99397)

In [72]:
prefix_string = 'How can'
new_df=new_df.reset_index(drop=True)
weights = new_df['Counts'].apply(lambda x: 1+ np.log1p(x)).values

In [73]:
# tranform the string using the tfidf model
tfidf_spelling = model_tf.transform([prefix_string])
# calculate cosine_matrix
cosine_similarity = linear_kernel(tfidf, tfidf_spelling)

In [74]:
#sort by order of similarity from 1 to 0:
similarity_scores = list(enumerate(cosine_similarity))
similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
similarity_scores = similarity_scores[0:10]

similarity_scores = [i for i in similarity_scores]
similarity_indices = [i[0] for i in similarity_scores]

In [77]:
#add weight to the potential results that had high frequency in orig data
for i in range(len(similarity_scores)):
  similarity_scores[i][1][0]=similarity_scores[i][1][0]*weights[similarity_indices][i]
  similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
  similarity_scores = similarity_scores[0:3]
  similarity_indices_w = [i[0] for i in similarity_scores]

In [78]:
print(prefix_string)
new_df.loc[similarity_indices_w]['Text'].tolist()

How can


['How can I help you today?', 'How can I help you?', 'How can we help you?']